1. Importamos las bibliotecas necesarias.

In [1]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import pandas as pd
import math
import random as rdm
import checkNNGradients as compararGradientes

In [2]:
# se pueden consultar las claves con data . keys ( )
weights = loadmat('C:/Users/HAO-H/Documents/Universidad/AA/zPracticas/practica4/ex4weights.mat')
data = loadmat('C:/Users/HAO-H/Documents/Universidad/AA/zPracticas/practica4/ex4data1.mat')
#Theta1 es de dimensión 25 x 401
#Theta2 es de dimensión 10 x 26
thet1, thet2 = weights['Theta1'], weights['Theta2']

In [3]:
# Valores de X e y
X = data ['X']
y = data ['y']
#params_rn = (np.random.random(size=25 * (400 + 1) + 10* (25 + 1)) - 0.5) * 0.25
#Union de thetas
params_rn=np.hstack((thet1.ravel(), thet2.ravel()))
#params_rn=np.hstack((thet1.ravel(order='F'), thet2.ravel(order='F')))

In [4]:
#función sigmoide
def s(z):
    return 1/(1 + np.exp(-z))

In [5]:
def propagacion_delante(X, y, Theta1, Theta2):
    filas = len(y)
    a1_ = np.insert(X, 0, values=np.ones(filas), axis=1)
    z2_ = a1_ @ Theta1.T
    a2_ =  np.insert(s(z2_), 0, values=np.ones(filas), axis=1)
    z3_ = a2_ @ Theta2.T
    h_ = s(z3_)
    return a1_ , z2_ , a2_ , z3_ , h_

In [6]:
def listToArray(y):
    if type(y) == list:
        y = np.array(y).reshape(-1, 1)
        
    return y

 La función de coste.

In [7]:
def fun_cost(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y_list_, reg):
    [m,n] = X.shape
    theta1= np.reshape(params_rn[:num_ocultas * (num_entradas+1)],
                        (num_ocultas, (num_entradas + 1)))
    theta2 = np.reshape(params_rn[(num_ocultas * (num_entradas + 1)):],
                        (num_etiquetas,(num_ocultas + 1)))
    
   
    a1, z2, a2, z3, h = propagacion_delante(X, y_list_, theta1, theta2)
    y_list_ = listToArray(y_list_)
    y_aux = pd.get_dummies(y_list_.flatten())
    
    #Función de coste regularizada
    primera = np.sum((np.sum(np.multiply(y_aux, np.log(h)) + np.multiply(1-y_aux,np.log(1-h))))/(-m))
    result = primera + reg * ((np.sum(np.sum(np.power(theta1[:,1:], 2),axis=1))) + 
                               (np.sum(np.sum(np.power(theta2[:,1:], 2), axis=1))))/(2*m)
    
    return result

In [8]:
fun_cost(params_rn, 400, 25, 10, X, y, 1)

0.3837698590909235

2. Calculo del gradiente

In [9]:
#funcion auxiliar para retro-propagacion: derivada de la funcion sigmoide
def g_der(z):
    return np.multiply(s(z),(1-s(z)))

In [10]:
#funcion para elegir E inicial segun los L_in L_out
def e_ini(L_in, L_out):
    return math.sqrt(6)/math.sqrt(L_in+L_out)

In [11]:
#inicializar una matriz de pesos theta, rdm = biblioteca.random
def pesosAleatorios(L_in, L_out):
    i = 0
    lista = []
    e = e_ini(L_in, L_out)
    while(i != L_out*(1+L_in)):
        i = i + 1
        lista.append(rdm(-e, e))
    return np.array(lista).resize(L_out, 1+L_in)

In [37]:
#gradiente sin regularizar
def retro_propagacion(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y_list_, reg):
    [m,n] = X.shape
    theta1= np.reshape(params_rn[:num_ocultas * (num_entradas+1)],
                        (num_ocultas, (num_entradas + 1)))
    theta2 = np.reshape(params_rn[(num_ocultas * (num_entradas + 1)):],
                        (num_etiquetas,(num_ocultas + 1)))
    
    a1, z2, a2, z3, h = propagacion_delante(X, y_list_, theta1, theta2) #1
    y_list_ = listToArray(y_list_)
    y_aux = pd.get_dummies(y_list_.flatten())
    #y_list = np.matrix(y_list)
    
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    
    for i in range(m):
        ones = np.ones(1)
        a1 = np.hstack((ones, X[i]))
        z2 = a1 @ theta1.T
        a2 = np.hstack((ones, s(z2)))
        z3 = a2 @ theta2.T
        a3 = s(z3)

        d3 = a3 - y_aux.iloc[i,:][np.newaxis,:]
        z2 = np.hstack((ones, z2))
        d2 = np.multiply(theta2.T @ d3.T, g_der(z2).T[:,np.newaxis])
        delta1 = delta1 + d2[1:,:] @ a1[np.newaxis,:]
        delta2 = delta2 + d3.T @ a2[np.newaxis,:]
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    #gradientes regularizadas
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * reg) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * reg) / m
    
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return grad

In [13]:
def backprop(params_rn , num_entradas , num_ocultas , num_etiquetas , X, y , reg):
    return fun_cost(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y, reg), retro_propagacion(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y, reg) 

In [38]:
compararGradientes.checkNNGradients(backprop, 0)

array([ 5.27761029e-11, -3.29852898e-13,  5.67279904e-12,  6.95585256e-12,
       -6.52669585e-11,  2.08456863e-12, -1.51965315e-11, -4.60273486e-11,
       -9.52194434e-11,  7.04842781e-12, -4.42525599e-11, -1.24605798e-10,
       -2.40059500e-11,  2.76547969e-12, -6.02735570e-12, -2.49761462e-11,
        2.37940986e-11, -4.96176017e-13,  9.77740458e-12,  2.73879391e-11,
        6.03759820e-11,  1.55131186e-11,  6.81166235e-12,  7.48807960e-12,
        1.90087945e-11,  1.88440930e-11,  7.15513759e-11,  1.33875688e-11,
        4.89146224e-12,  1.15286669e-11,  1.70987668e-11,  2.23745744e-11,
        7.55119856e-11,  1.60134683e-11,  6.39788222e-12,  1.78091986e-11,
        1.43913215e-11,  2.04546102e-11])